In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

from multiprocessing import Process, Manager, Semaphore
import multiprocessing

import tensorflow as tf

import GPyOpt

import time

import numpy as np

import utils.losses_utils as losses

import utils.data_utils as data_utils

import utils.fmri_utils as fmri_utils

import utils.imputation_utils as imputation

import utils.training_utils as training

import utils.layers_utils as layers

import matplotlib.pyplot as plt

import gc

from pathlib import Path

import pickle

home = str(Path.home())

dataset="02"#"01", "02"

bold_shift=3
f_resample=1.8
fmri_resolution_factor=6
n_partitions=24
by_partitions=False
partition_length=14
n_voxels = 100
missing_values_mode = "region"#"region", "time", "volume"
rm_rate = 0.10

cpu_number = multiprocessing.cpu_count()
cpu_usage = 0.50
concurrency = int(cpu_number*cpu_usage)

device = "CPU"

gpu = tf.config.experimental.list_physical_devices(device)[0]

if(device == "CPU"):
    devices = tf.config.experimental.list_physical_devices(device_type='CPU')
    tf.config.experimental.set_visible_devices(devices=devices, device_type='CPU')
    tf.config.set_soft_device_placement(True)
    tf.config.log_device_placement=True
    
else:
    tf.config.set_soft_device_placement(True)
    tf.config.log_device_placement=True
    tf.config.experimental.set_memory_growth(gpu, True)

    tf.config.experimental.set_virtual_device_configuration(
            gpu,
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

## Loading Data

In [2]:
if(dataset=="01"):
    instances = list(range(10))
elif(dataset=="02"):
    instances = list(range(14))
    
_, _bold_set, mask, scalers = data_utils.load_data(instances, 
                                                    n_voxels=None, 
                                                    bold_shift=bold_shift, 
                                                    n_partitions=n_partitions, 
                                                    by_partitions=by_partitions, 
                                                    partition_length=partition_length, 
                                                    f_resample=f_resample, 
                                                    fmri_resolution_factor=fmri_resolution_factor, 
                                                    task=1, #1-auditory;2-visual
                                                    run=1,
                                                    standardize_eeg=True, 
                                                    standardize_fmri=True, 
                                                    dataset=dataset)

coords = np.argwhere(mask.mask_img_.get_data() == 1.0)

if(dataset=="01"):
    n_individuals=10
    n_individuals_train = 8
    n_individuals_test = 2

    n_frames = _bold_set.shape[2]

    _bold_set = _bold_set[:,:n_voxels]
    _bold_set = _bold_set.reshape(_bold_set.shape + (1,))
    
    min_train = np.amin(_bold_set)
    max_value = np.amin([min_train])
    _bold_set = _bold_set - max_value+0.001

    _bold_set = np.log(_bold_set)
    _bold_set = _bold_set.astype('float32')
    
    coords = coords[:n_voxels]
    
elif(dataset=="02"):
    n_individuals=14
    
    n_individuals_train = 10
    n_individuals_test = 4
    
    n_frames = _bold_set.shape[2]
    
    _bold_set = _bold_set.reshape(_bold_set.shape + (1,))
    _bold_set = _bold_set[:,:n_voxels]
    _bold_set = _bold_set.astype('float32')
    
    coords = coords[:n_voxels]
    
if(not by_partitions):
    n_partitions=int(_bold_set.shape[0]/n_individuals)

(196, 49, 111, 14)
(196, 100, 14)


## Train Test Split

In [3]:
bold_train = _bold_set[:n_partitions*n_individuals_train]
bold_test = _bold_set[n_partitions*n_individuals_train:]

bold_train = data_utils.scale_voxels(bold_train)
bold_test = data_utils.scale_voxels(bold_test)

## Load Correlation Matrix

In [4]:
train_corr_matrix = imputation.compute_corr_matrix(bold_train, n_individuals=n_individuals_train, n_partitions=n_partitions)
train_corr_matrix = train_corr_matrix[:n_voxels,:n_voxels]

## Preprocessing Data

In [5]:
shuffled_bold_train = data_utils.shuffle_individuals_batch(bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train, n_frames=n_frames)
shuffled_bold_test = data_utils.shuffle_individuals_batch(bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test, n_frames=n_frames)

shuffled_bold_train = data_utils.decompress_maintain_dim(shuffled_bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train)
shuffled_bold_test = data_utils.decompress_maintain_dim(shuffled_bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test)

## Randomly Remove Values

In [6]:
if(missing_values_mode == "volume"):
    missing_values_bold_train = imputation.random_missing_values_volume(shuffled_bold_train, rm_pct=rm_rate)
    missing_values_bold_test = imputation.random_missing_values_volume(shuffled_bold_test, rm_pct=rm_rate)
    denoising_missing_values_bold_train = imputation.random_missing_values_volume(bold_train, rm_pct=rm_rate)
    denoising_missing_values_bold_test = imputation.random_missing_values_volume(bold_test, rm_pct=rm_rate)
elif(missing_values_mode == "time"):
    missing_values_bold_train = imputation.random_missing_values_time(shuffled_bold_train, rm_pct=rm_rate)
    missing_values_bold_test = imputation.random_missing_values_time(shuffled_bold_test, rm_pct=rm_rate)
    denoising_missing_values_bold_train = imputation.random_missing_values_time(bold_train, rm_pct=rm_rate)
    denoising_missing_values_bold_test = imputation.random_missing_values_time(bold_test, rm_pct=rm_rate)
elif(missing_values_mode == "region"):
    missing_values_bold_train = imputation.region_missing_values_time(shuffled_bold_train, coords, rm_pct=rm_rate)    
    missing_values_bold_test = imputation.region_missing_values_time(shuffled_bold_test, coords, rm_pct=rm_rate)
    denoising_missing_values_bold_train = imputation.region_missing_values_time(bold_train, coords, rm_pct=rm_rate)
    denoising_missing_values_bold_test = imputation.region_missing_values_time(bold_test, coords, rm_pct=rm_rate)

## Compute Masks and reshape data

In [7]:
missing_values_bold_train = data_utils.compress_maintain_dim(missing_values_bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train)
missing_values_bold_test = data_utils.compress_maintain_dim(missing_values_bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test)
denoising_missing_values_bold_train = data_utils.compress_maintain_dim(denoising_missing_values_bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train)
denoising_missing_values_bold_test = data_utils.compress_maintain_dim(denoising_missing_values_bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test)

bold_missing_with_mask_train = imputation.dataset_mask_missing_values(missing_values_bold_train)
bold_missing_with_mask_test = imputation.dataset_mask_missing_values(missing_values_bold_test)
target_missing_train = data_utils.compress_maintain_dim(shuffled_bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train)
target_missing_test = data_utils.compress_maintain_dim(shuffled_bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test)
denoising_bold_missing_with_mask_train = imputation.dataset_mask_missing_values(denoising_missing_values_bold_train)
denoising_bold_missing_with_mask_test = imputation.dataset_mask_missing_values(denoising_missing_values_bold_test)
target_bold_train = data_utils.compress_maintain_dim(bold_train, n_partitions=n_partitions, n_individuals=n_individuals_train)
target_bold_test = data_utils.compress_maintain_dim(bold_test, n_partitions=n_partitions, n_individuals=n_individuals_test)

bold_missing_with_mask_train = bold_missing_with_mask_train.astype('float32')
bold_missing_with_mask_test = bold_missing_with_mask_test.astype('float32')
target_missing_train = target_missing_train.astype('float32')
target_missing_test = target_missing_test.astype('float32')
denoising_bold_missing_with_mask_train = denoising_bold_missing_with_mask_train.astype('float32')
denoising_bold_missing_with_mask_test = denoising_bold_missing_with_mask_test.astype('float32')
target_bold_train = target_bold_train.astype('float32')
target_bold_test = target_bold_test.astype('float32')

target_missing_train = target_missing_train.reshape(target_missing_train.shape[:-1])
target_missing_test = target_missing_test.reshape(target_missing_test.shape[:-1])

## Built Iterative only Spatial Model

In [8]:
hyperparameters = np.load(home+"/fmri_imputation/hyperparameters/01_chained_spatial_only_hyperparameters.npy")

current_epochs_chained_spatial_only=int(hyperparameters[0])
current_learning_rate=float(hyperparameters[1])
current_batch_size=14
current_optimizer_chained_spatial_only = tf.keras.optimizers.Adamax(learning_rate=current_learning_rate)

chained_spatial_only = layers.RecursiveMLPBlock(target_missing_train.shape[1], corr_matrix=train_corr_matrix, activation=tf.keras.activations.linear)
chained_spatial_only.build(input_shape=(1, target_missing_train.shape[1]))

## Built Dropout only Spatial Model

In [9]:
hyperparameters = np.load(home+"/fmri_imputation/hyperparameters/01_dropout_spatial_only_hyperparameters.npy")

current_epochs_dropout_spatial_only=int(hyperparameters[0])
current_learning_rate=float(hyperparameters[1])
current_batch_size=14
current_optimizer_dropout_spatial_only = tf.keras.optimizers.Adamax(learning_rate=current_learning_rate)

dropout_spatial_only = layers.DropoutMLPBlock(target_missing_train.shape[1], activation=tf.keras.activations.linear)
dropout_spatial_only.build(input_shape=(1, target_missing_train.shape[1]))

## Build Iterative Model

In [10]:
activation_funtion = tf.keras.activations.linear

hyperparameters = np.load(home+"/fmri_imputation/hyperparameters/01_chained_joint_hyperparameters.npy")

current_epochs_spatial_chained=int(hyperparameters[0])
current_epochs_denoiser_chained=int(hyperparameters[1])
current_epochs_iterations_chained=int(hyperparameters[2])
current_learning_rate_spatial_chained=float(hyperparameters[3])
current_learning_rate_denoiser_chained=float(hyperparameters[4])
current_denoiser_reg_chained=float(hyperparameters[5])
current_use_bias_chained=bool(hyperparameters[6])
current_dropout_chained=float(hyperparameters[7])
current_recurrent_dropout_chained=float(hyperparameters[8])
current_batch_size=14
current_optimizer_spatial_chained = tf.keras.optimizers.Adamax(learning_rate=current_learning_rate_spatial_chained)
current_optimizer_denoiser_chained = tf.keras.optimizers.Adam(learning_rate=current_learning_rate_denoiser_chained)

iterative = layers.RecursiveMLPBlock(target_missing_train.shape[1], corr_matrix=train_corr_matrix, activation=activation_funtion)
iterative.build(input_shape=(1, target_missing_train.shape[1]))

denoiser_chained = tf.keras.Sequential()
denoiser_chained.add(tf.keras.layers.GRU(1, kernel_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_chained),
                                    recurrent_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_chained),
                                    bias_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_chained),
                                    return_sequences=True, input_shape=_bold_set.shape[2:],
                                    use_bias=current_use_bias_chained, dropout=current_dropout_chained, 
                                    recurrent_dropout=current_recurrent_dropout_chained))
denoiser_chained.build(input_shape=_bold_set.shape[2:])

## Build Dropout Model

In [11]:
activation_funtion = tf.keras.activations.linear

hyperparameters = np.load(home+"/fmri_imputation/hyperparameters/01_dropout_joint_hyperparameters.npy")

current_epochs_spatial_dropout=int(hyperparameters[0])
current_epochs_denoiser_dropout=int(hyperparameters[1])
current_epochs_iterations_dropout=int(hyperparameters[2])
current_learning_rate_spatial_dropout=float(hyperparameters[3])
current_learning_rate_denoiser_dropout=float(hyperparameters[4])
current_denoiser_reg_dropout=float(hyperparameters[5])
current_use_bias_dropout=bool(hyperparameters[6])
current_dropout_dropout=float(hyperparameters[7])
current_recurrent_dropout_dropout=float(hyperparameters[8])
current_batch_size=14
current_optimizer_spatial_dropout = tf.keras.optimizers.Adamax(learning_rate=current_learning_rate_spatial_dropout)
current_optimizer_denoiser_dropout = tf.keras.optimizers.Adam(learning_rate=current_learning_rate_denoiser_dropout)

dropout = layers.DropoutMLPBlock(target_missing_train.shape[1], activation=activation_funtion)
dropout.build(input_shape=(1, target_missing_train.shape[1]))

denoiser_dropout = tf.keras.Sequential()
denoiser_dropout.add(tf.keras.layers.GRU(1, kernel_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_dropout),
                                    recurrent_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_dropout),
                                    bias_regularizer=tf.keras.regularizers.l2(current_denoiser_reg_dropout),
                                    return_sequences=True, input_shape=_bold_set.shape[2:],
                                    use_bias=current_use_bias_dropout, dropout=current_dropout_dropout, 
                                    recurrent_dropout=current_recurrent_dropout_dropout))
denoiser_dropout.build(input_shape=_bold_set.shape[2:])

## Build Context Encoder

In [12]:
#ce_bold_missing_with_mask_train = bold_missing_with_mask_train.reshape(bold_missing_with_mask_train.shape + (1,))
#ce_target_missing_train = target_missing_train.reshape(target_missing_train.shape + (1,))

#ce_bold_missing_with_mask_test = bold_missing_with_mask_test.reshape(bold_missing_with_mask_test.shape + (1,))
#ce_target_missing_test = target_missing_test.reshape(target_missing_test.shape + (1,))

#ce_model = tf.keras.Sequential()

#ce_model.add(tf.keras.layers.Conv1D(1, kernel_size=(10), strides=(2), input_shape=ce_target_missing_train.shape[1:]))
#ce_model.add(tf.keras.layers.UpSampling1D(size=5.0))
#ce_model.add(tf.keras.layers.Conv1D(1, kernel_size=(31), strides=(2)))

#ce_model.build(input_shape=ce_target_missing_train.shape)

## Build Mean Imputation from Training Set

In [13]:
mean_imputation = imputation.Mean_Imputation(bold_missing_with_mask_train)

## Build kNN Imputation from Coordinates

In [14]:
knn = imputation.knn_imputation(coords)

## Train Models

In [15]:
training.imputation_training(chained_spatial_only, bold_missing_with_mask_train, target_missing_train, missing_mask_val=None, target_missing_val=None,optimizer=current_optimizer_chained_spatial_only, batch_size=current_batch_size, n_epochs=current_epochs_chained_spatial_only, verbose=True)

training.imputation_training(dropout_spatial_only, bold_missing_with_mask_train, target_missing_train, missing_mask_val=None, target_missing_val=None,optimizer=current_optimizer_dropout_spatial_only, batch_size=current_batch_size, n_epochs=current_epochs_dropout_spatial_only, verbose=True)

training.alternate_training_spatial_denoiser(iterative, denoiser_chained, bold_missing_with_mask_train, target_missing_train, None, None, denoising_bold_missing_with_mask_train, None, target_bold_train, None,epochs_iterations=current_epochs_iterations_chained, optimizer_spatial=current_optimizer_spatial_chained, optimizer_denoiser=current_optimizer_denoiser_chained, batch_size=current_batch_size, epochs_spatial=current_epochs_spatial_chained, epochs_denoiser=current_epochs_denoiser_chained, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals_train=n_individuals_train,verbose=True)

training.alternate_training_spatial_denoiser(dropout, denoiser_dropout, bold_missing_with_mask_train, target_missing_train, None, None, denoising_bold_missing_with_mask_train, None, target_bold_train, None,epochs_iterations=current_epochs_iterations_dropout, optimizer_spatial=current_optimizer_spatial_dropout, optimizer_denoiser=current_optimizer_denoiser_dropout, batch_size=current_batch_size, epochs_spatial=current_epochs_spatial_dropout, epochs_denoiser=current_epochs_denoiser_dropout, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals_train=n_individuals_train,verbose=True)

#ce_results = training.ce_training(ce_model, ce_bold_missing_with_mask_train, ce_target_missing_train, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0001), batch_size=14, n_epochs=20)

Epoch 1 Average Loss Retrieval Train: 0.8716230311564037
Epoch 1 Average Loss Complete Train: 0.9127404532262258
Epoch 2 Average Loss Retrieval Train: 0.8348909910236086
Epoch 2 Average Loss Complete Train: 0.889430022239685
Epoch 1 Average Loss Retrieval Train: 0.8738270027296884
Epoch 1 Average Loss Complete Train: 0.872578547682081
Epoch 2 Average Loss Retrieval Train: 0.8401124664715358
Epoch 2 Average Loss Complete Train: 0.8549553449664797
Epoch 1 Average Loss Retrieval Train: 0.8670862061636788
Epoch 1 Average Loss Complete Train: 0.9031357880149569
Epoch 2 Average Loss Retrieval Train: 0.8078877440520695
Epoch 2 Average Loss Complete Train: 0.8674484418971198
Epoch 3 Average Loss Retrieval Train: 0.7709479868412018
Epoch 3 Average Loss Complete Train: 0.8502533346414566
Epoch 4 Average Loss Retrieval Train: 0.7428950079849788
Epoch 4 Average Loss Complete Train: 0.8391434354441506
Epoch 1  with train loss: 0.8146643924713135
Epoch 2  with train loss: 0.8035518300533294
Epoch 1 

### Get Outputs from Neural Network Models

In [16]:
out_chained_spatial_only_test = data_utils.out_by_batch(chained_spatial_only, bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels)
out_dropout_spatial_only_test = data_utils.out_by_batch(dropout_spatial_only, bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels)

imputted_bold_test, target_denoised_test = data_utils.format_to_denoiser(data_utils.out_by_batch(dropout, denoising_bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels), denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
dropout_denoised_test, dropout_target_test = data_utils.format_to_mae_loss(denoiser_dropout(imputted_bold_test).numpy(), target_denoised_test, n_individuals=n_individuals_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions)

imputted_bold_test, target_denoised_test = data_utils.format_to_denoiser(data_utils.out_by_batch(iterative, denoising_bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels), denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
chained_denoised_test, chained_target_test = data_utils.format_to_mae_loss(denoiser_chained(imputted_bold_test).numpy(), target_denoised_test, n_individuals=n_individuals_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions)

#out_ce_train = ce_model(ce_bold_missing_with_mask_train[:,0,:]).numpy()
#out_ce_test = ce_model(ce_bold_missing_with_mask_test[:,0,:]).numpy()

#out_ce_train = out_ce_train.reshape(out_ce_train.shape[:-1])
#out_ce_test = out_ce_test.reshape(out_ce_test.shape[:-1])

In [ ]:
x = data_utils.deshuflle_individuals_batch(chained_denoised_test, 
                                n_individuals=n_individuals_test,
                                n_partitions=n_partitions,
                                n_frames=_bold_set.shape[2])
print(x.shape)




view_imputted_bold = bold_missing_with_mask_test[:,0,:].copy()

for instance in range(x.shape[0]):
    
    i = 0
    #print(view_imputted_bold[instance])
    print(np.sum(bold_missing_with_mask_test[instance, 1, :]))
    for voxel in range(bold_missing_with_mask_test.shape[2]):
        if(bold_missing_with_mask_test[instance,1,voxel]):
            #print(i)
            #view_imputted_bold[instance, voxel] = x[instance, i]
            i+=1
    #print(view_imputted_bold[instance])
    
    
x = data_utils.decompress_maintain_dim(x.reshape(x.shape +(1,)), n_partitions=n_partitions, n_individuals=n_individuals_test)
print(x.shape)

print(bold_missing_with_mask_test.shape)



## Compute Mean Imputation

In [17]:
out_mean_train = mean_imputation.imputation_batch(bold_missing_with_mask_train)
out_mean_test = mean_imputation.imputation_batch(bold_missing_with_mask_test)

## Compute kNN regressor

In [18]:
out_knn_test = knn.imputation_batch(bold_missing_with_mask_test, k=10)

## Compute Barycenter Imputation

In [19]:
out_barycenter_test = imputation.barycenter_imputation(imputation.random_missing_values_time(shuffled_bold_test, rm_pct=rm_rate), n_partitions=n_partitions, n_individuals=n_individuals_test)

## Compute MICE Imputation

In [20]:
out_mice_test = imputation.mice_imputation(bold_missing_with_mask_train, bold_missing_with_mask_test)

## Spatial Results

### MAE

In [21]:
loss_knn_val = losses.compute_mae_loss(out_knn_test, target_missing_test, bold_missing_with_mask_test, model_name="kNN", set_name="test")

loss_barycenter_val = losses.compute_mae_loss(out_barycenter_test, target_missing_test, bold_missing_with_mask_test, model_name="Barycenter", set_name="test")

loss_mean_val = losses.compute_mae_loss(out_mean_test, target_missing_test, bold_missing_with_mask_test, model_name="mean", set_name="test")

#loss_ce_val = losses.compute_mae_loss(out_ce_test, target_missing_test, bold_missing_with_mask_test, model_name="ce", set_name="test")

loss_mice_val = losses.compute_mae_loss(out_mice_test, target_missing_test, bold_missing_with_mask_test, model_name="MICE", set_name="test")

loss_dropout_spatial_val = losses.compute_mae_loss(out_dropout_spatial_only_test, target_missing_test, bold_missing_with_mask_test, model_name="Dropout Spatial Only", set_name="test")

loss_dropout_val = losses.compute_mae_loss_maskless(dropout_denoised_test, dropout_target_test, model_name="dropout", set_name="test")

loss_chained_spatial_val = losses.compute_mae_loss(out_chained_spatial_only_test, target_missing_test, bold_missing_with_mask_test, model_name="Chained Spatial Only", set_name="test")

loss_iterative_val = losses.compute_mae_loss_maskless(chained_denoised_test, chained_target_test, model_name="iterative", set_name="test")

kNN $0.84\\pm2.61$
Barycenter $1.02\\pm3.16$
mean $0.81\\pm2.50$
MICE $0.82\\pm2.53$
Dropout Spatial Only $10.71\\pm30.60$
dropout $0.82\\pm0.60$
Chained Spatial Only $10.71\\pm30.60$
iterative $0.83\\pm0.60$


### RMSE

In [22]:
loss_knn_val = losses.compute_mse_loss(out_knn_test, target_missing_test, bold_missing_with_mask_test, model_name="kNN", set_name="test", root=True)

loss_barycenter_val = losses.compute_mse_loss(out_barycenter_test, target_missing_test, bold_missing_with_mask_test, model_name="Barycenter", set_name="test", root=True)

loss_mean_val = losses.compute_mse_loss(out_mean_test, target_missing_test, bold_missing_with_mask_test, model_name="mean", set_name="test", root=True)

#loss_ce_val = losses.compute_mse_loss(out_ce_test, target_missing_test, bold_missing_with_mask_test, model_name="ce", set_name="test", root=True)

loss_mice_val = losses.compute_mse_loss(out_mice_test, target_missing_test, bold_missing_with_mask_test, model_name="MICE", set_name="test", root=True)

loss_dropout_spatial_val = losses.compute_mse_loss(out_dropout_spatial_only_test, target_missing_test, bold_missing_with_mask_test, model_name="Dropout Spatial Only", set_name="test", root=True)

loss_dropout_val = losses.compute_mse_loss_maskless(dropout_denoised_test, dropout_target_test, model_name="dropout", set_name="test", root=True)

loss_chained_spatial_val = losses.compute_mse_loss(out_chained_spatial_only_test, target_missing_test, bold_missing_with_mask_test, model_name="Chained Spatial Only", set_name="test", root=True)

loss_iterative_val = losses.compute_mse_loss_maskless(chained_denoised_test, chained_target_test, model_name="iterative", set_name="test", root=True)

kNN $1.05\\pm3.23$
Barycenter $1.27\\pm3.90$
mean $1.01\\pm3.10$
MICE $1.02\\pm3.13$
Dropout Spatial Only $4.29\\pm10.52$
dropout $1.02\\pm0.63$
Chained Spatial Only $4.29\\pm10.52$
iterative $1.02\\pm0.63$


## Time Results

In [23]:
imputted_knn_bold_test, target_knn_test = data_utils.format_to_denoiser(out_knn_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_barycenter_bold_test, target_barycenter_test = data_utils.format_to_denoiser(out_barycenter_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_mean_bold_test, target_mean_test = data_utils.format_to_denoiser(out_mean_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
#imputted_ce_bold_test, target_ce_test = data_utils.format_to_denoiser(out_ce_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_mice_bold_test, target_mice_test = data_utils.format_to_denoiser(out_mice_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_dropout_spatial_only_bold_test, target_ce_test = data_utils.format_to_denoiser(out_dropout_spatial_only_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_chained_spatial_only_bold_test, target_ce_test = data_utils.format_to_denoiser(out_chained_spatial_only_test, denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_dropout_bold_test, target_dropout_test = data_utils.format_to_denoiser(data_utils.out_by_batch(dropout, denoising_bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels), denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)
imputted_iterative_bold_test, target_iterative_test = data_utils.format_to_denoiser(data_utils.out_by_batch(iterative, denoising_bold_missing_with_mask_test, batch_size=current_batch_size, n_voxels=n_voxels), denoising_bold_missing_with_mask_test, target_bold_test, rm_rate=rm_rate, n_voxels=n_voxels, n_partitions=n_partitions, n_individuals=n_individuals_test)

### MAE

In [24]:
loss_knn_val = losses.compute_mae_loss_maskless(imputted_knn_bold_test, target_knn_test, model_name="kNN", set_name="test")

loss_barycenter_val = losses.compute_mae_loss_maskless(imputted_barycenter_bold_test, target_barycenter_test, model_name="Barycenter", set_name="test")

loss_mean_val = losses.compute_mae_loss_maskless(imputted_mean_bold_test, target_mean_test, model_name="mean", set_name="test")

#loss_ce_val = losses.compute_mae_loss_maskless(imputted_ce_bold_test, target_ce_test, model_name="ce", set_name="test")

loss_mice_val = losses.compute_mae_loss_maskless(imputted_mice_bold_test, target_mice_test, model_name="MICE", set_name="test")

loss_dropout_spatial_only_val = losses.compute_mae_loss_maskless(imputted_dropout_spatial_only_bold_test, target_ce_test, model_name="Dropout Spatial Only", set_name="test")

loss_dropout_val = losses.compute_mae_loss_maskless(imputted_dropout_bold_test, target_dropout_test, model_name="dropout", set_name="test")

loss_chained_spatial_only_val = losses.compute_mae_loss_maskless(imputted_chained_spatial_only_bold_test, target_ce_test, model_name="Chained Spatial Only", set_name="test")

loss_iterative_val = losses.compute_mae_loss_maskless(imputted_iterative_bold_test, target_iterative_test, model_name="iterative", set_name="test")

kNN $1.10\\pm0.84$
Barycenter $1.12\\pm0.85$
mean $1.10\\pm0.84$
MICE $1.10\\pm0.84$
Dropout Spatial Only $1.09\\pm0.84$
dropout $1.10\\pm0.82$
Chained Spatial Only $1.09\\pm0.84$
iterative $1.10\\pm0.82$


### RMSE

In [25]:
loss_knn_val = losses.compute_mse_loss_maskless(imputted_knn_bold_test, target_knn_test, model_name="kNN", set_name="test", root=True)

loss_barycenter_val = losses.compute_mse_loss_maskless(imputted_barycenter_bold_test, target_barycenter_test, model_name="Barycenter", set_name="test", root=True)

loss_mean_val = losses.compute_mse_loss_maskless(imputted_mean_bold_test, target_mean_test, model_name="mean", set_name="test", root=True)

#loss_ce_val = losses.compute_mse_loss_maskless(imputted_ce_bold_test, target_ce_test, model_name="ce", set_name="test", root=True)

loss_mice_val = losses.compute_mse_loss_maskless(imputted_mice_bold_test, target_mice_test, model_name="MICE", set_name="test", root=True)

loss_dropout_spatial_only_val = losses.compute_mse_loss_maskless(imputted_dropout_spatial_only_bold_test, target_ce_test, model_name="Dropout Spatial Only", set_name="test", root=True)

loss_dropout_val = losses.compute_mse_loss_maskless(imputted_dropout_bold_test, target_dropout_test, model_name="dropout", set_name="test", root=True)

loss_chained_spatial_only_val = losses.compute_mse_loss_maskless(imputted_chained_spatial_only_bold_test, target_ce_test, model_name="Chained Spatial Only", set_name="test", root=True)

loss_iterative_val = losses.compute_mse_loss_maskless(imputted_iterative_bold_test, target_iterative_test, model_name="iterative", set_name="test", root=True)

kNN $1.39\\pm0.89$
Barycenter $1.40\\pm0.89$
mean $1.38\\pm0.89$
MICE $1.38\\pm0.88$
Dropout Spatial Only $1.37\\pm0.88$
dropout $1.37\\pm0.87$
Chained Spatial Only $1.37\\pm0.88$
iterative $1.37\\pm0.87$
